In [1]:
"""
Command line tool for executing GEMMA

Authors:
Anna G. Green
"""
import pandas as pd
import subprocess
import numpy as np
import ruamel.yaml as yaml
import sys
from evcouplings.utils import valid_file, verify_resources, run
from evcouplings.align import Alignment
import os
from copy import deepcopy
from scipy import stats

ALPHABET_NOGAP = "ACGT"
GAP_CHAR = "-"
ALPHABET = GAP_CHAR + ALPHABET_NOGAP

genotype_translator = {
    0:"A", 1:"C", 2:"G", 3:"T", 9:"-"
}
genotype_translator_r = {y:x for x,y in genotype_translator.items()}
def map_matrix(matrix, map_):
    """
    Map elements in a numpy array using alphabet
    Parameters
    ----------
    matrix : np.array
        Matrix that should be remapped
    map_ : defaultdict
        Map that will be applied to matrix elements
    Returns
    -------
    np.array
        Remapped matrix
    """
    return np.vectorize(map_.__getitem__)(matrix)


def calculate_frequencies(matrix, num_symbols):
    """
    Calculate single-site frequencies of symbols in alignment
    Parameters
    ----------
    matrix : np.array
        N x L matrix containing N sequences of length L.
        Matrix must be mapped to range(0, num_symbols) using
        map_matrix function
    num_symbols : int
        Number of different symbols contained in alignment
    Returns
    -------
    np.array
        Matrix of size L x num_symbols containing relative
        column frequencies of all characters
    """
    N, L = matrix.shape
    fi = np.zeros((L, num_symbols))
    for s in range(N):
        for i in range(L):
            fi[i, matrix[s, i]] += 1
    return np.divide(fi, N)

class GenotypeMatrix:
    """
    Takes inspiration from the evcouplings python package Alignment class
    """
    def __init__(self, strain_list, position_list, genotype_data=None, matrix_data=None):

        self.positions = position_list
        self.position_to_index = {p:idx for idx, p in enumerate(self.positions)}
        self.strains = strain_list
        self.strain_to_index = {p:idx for idx, p in enumerate(self.strains)}

        # If user provided genotype data, do some sanity checks
        self.genotype_data = genotype_data
        # Make sure that every position and every genotype is represented in the genotype_data
        if not self.genotype_data is None:
            assert set(self.positions).issubset(genotype_data.POS)
            assert set(self.strains).issubset(genotype_data.STRAIN)

        # if user provided matrix data, set it up
        if not matrix_data is None:
            assert matrix_data.shape[0] == len(self.strains)
            assert matrix_data.shape[1] == len(self.positions)
            self.matrix = matrix_data
            self.reference_alleles = deepcopy(self.matrix[0, :])
            
            if 0 in matrix_data:
                self.matrix_mapped = matrix_data
            else:
                self.matrix_mapped = None
            
        else:
            self.matrix = np.zeros((len(self.strains), len(self.positions)), dtype=str)
            self.reference_alleles = None
            self.matrix_mapped = None

        self.alphabet_map = {
            c: i for i, c in enumerate(ALPHABET)
        }
        self.num_symbols = len(self.alphabet_map)
        
        self._frequencies = None
        self._major_alleles = None
        

    @classmethod
    def from_df(cls, file):
        """
        Initializes a GenotypeMatrix using an input pd.DataFrame, where rows = strains and columns=positions, and
        the data will become the contents of self.matrix
        """
        df = pd.read_csv(file, index_col=0, header=0)

        return cls(
            list(df.index), list(df.columns), genotype_data=None, matrix_data=df.values
        )

    def make_matrix(self):
        '''
        Populates the matrix of strains by positions with genotype data from
        the input genotype data file via the following heuristic:

        0. Initialize every position in every genome as the reference allele
        1. For each position in each strain, determine if there is any information
            in the genotype data input. If not, leave that position as Reference. If yes,
            proceed to step 2.
        2. If the alternate allele is an insertion or a non-standard character, insert a gap
            at that position.
        3. If the alternate allele did not pass quality threshold (determined by QUAL flag being 0),
            insert a gap at that position
        4. If alternate allele is a SNP and is not low quality, replace reference with alternate allale

        Returns
        -------
        np.ndarray:
            an N_strains x N_positions array of strings
        '''
        print("preparing the reference")

        position_subset = self.genotype_data.query("POS in @self.positions")

        position_groups = position_subset.groupby("POS")
        for position, subset in position_groups:
            allele = subset.loc[subset.index[0], "REF"]
            self.matrix[:, self.position_to_index[position]] = allele

        self.reference_alleles = deepcopy(self.matrix[0, :])
        #print("assigned the reference", self.reference_alleles)

        # iterate through each strain
        print("filling in the isolate genotypes")
        strain_group = position_subset.query("STRAIN in @self.strains").groupby("STRAIN")

        for strain, subset in strain_group:

            # iterate through each position
            position_groups = subset.groupby("POS")
            for position, subsubset in position_groups:

                # if we didn't find any information for this position, leave as reference allele
                if len(subsubset) == 0:
                    continue

                subsubset = subsubset.iloc[0,:]

                # If we did find information but its an indel, treat as missing
                if len(subsubset.ALT) > 1 or  not subsubset.ALT in ALPHABET:
                    self.matrix[self.strain_to_index[strain],self.position_to_index[position]] = "-"

                # If we found information but it didn't pass quality filter, treat as missing
                elif subsubset.QUAL==0:
                    self.matrix[self.strain_to_index[strain],self.position_to_index[position]] = "-"

                # else, change to alterante allele
                else:
                    self.matrix[self.strain_to_index[strain],self.position_to_index[position]] = subsubset.ALT


    def __ensure_mapped_matrix(self):
        """
        Ensure self.matrix_mapped exists
        """
        if self.matrix_mapped is None:
            self.matrix_mapped = map_matrix(
                self.matrix, self.alphabet_map
            )

    @property
    def major_alleles(self):
        """
        Returns/calculates identity of major allele for each position in alignment.
        Also sets self._major_alleles member variable for later reuse.

        Returns
        -------
        np.array
            Reference to self._major_alleles
        """
        if self._major_alleles is None:

            self._major_alleles = np.argmax(
                self.frequencies, axis=1
            )

        return self._major_alleles

    @property
    def frequencies(self):
        """
        Returns/calculates single-site frequencies of symbols in alignment.
        Also sets self._frequencies member variable for later reuse.

        Returns
        -------
        np.array
            Reference to self._frequencies
        """
        if self._frequencies is None:
            self.__ensure_mapped_matrix()

            self._frequencies = calculate_frequencies(
                self.matrix_mapped, self.num_symbols
            )

        return self._frequencies

    def write_fasta(self, fileobj, strain_list=None):
        """
        Creates a fasta-formatted file of pseudo-sequences for strains and positions in self.matrix

        Parameters
        ----------
        fileobj: filehandle
            fileobject to which to write
        strain_list: list of str, optional (default=None)
            List of strain names, in order, which will be written to file. If None,
            will write all strains in self.strains

        """
        if strain_list is None:
            strain_list = self.strains

        for strain in strain_list:
            seq = "".join(self.matrix[self.strain_to_index[strain],:])
            fileobj.write(f">{strain}\n{seq}\n")

    def write_GEMMA(self, fileobj, positions=None, major_allele_string="\"A\"", minor_allele_string="TRUE",
        major_allele_code='0', minor_allele_code='1',gap_code="NA"):
        """
        Creates a file of loci in correct input format for GEMMA

        User warning: Changing the optional variables to values other than their defaults may result in errors when
        using the program GEMMA. Verify that all inputs are still valid before proceeding.

        Gemma input format:
        position_name,major_allele_string,minor_allele_string,{allele code for strain 1, ..., allele code for strain N}

        Parameters
        ----------
        fileobj: filehandle
            file to which to write
        positions: list of str, optional (default: None)
            specify subset of positions to write. If None, use all positions
        major_allele_string: str, optional (default "A")
            string used to represent the major allele
        minor_allele_string: str, optional (default "T")
            string used to represent the minor allele
        major_allele_code: str, optional (default "1")
            string used to code for the major allele in the input
        minor_allele_code: str, optional (default "0")
            string used to code for the major allele in the input
        gap_code: str, optional (default "NaN")
            string used to code for the gap/missing data in the input

        """

        if positions is None:
            positions = self.positions

        self.__ensure_mapped_matrix()
        if not 0 in self.reference_alleles:
            reference_alleles = [self.alphabet_map[x] for x in self.reference_alleles]
        else:
            reference_alleles = self.reference_alleles
        print("calculating reference allele matrix with gap code", gap_code)  
        major_allele_matrix = calculate_reference_allele_matrix_for_pair_regression(
            self.matrix_mapped,
            reference_alleles,
            encode_gaps=True,
            gap_code=gap_code,
            ref_allele_code=major_allele_code,
            non_ref_allele_code=minor_allele_code
        )

        for position in positions:
            # slice corresponding to all strains
            vec = major_allele_matrix[:, self.position_to_index[position]]

            fileobj.write(f'{position},{major_allele_string},{minor_allele_string},{",".join(vec)}\n')

    def to_df(self):
        """
        Creates a pd.DataFrame, with rows corresponding to strains, columns corresponding to positions,
        and entries corresponding to the alleles in self.matrix
        Returns
        -------
        pd.DataFrame
        """
        df = pd.DataFrame(self.matrix, index=self.strains, columns=self.positions)
        return df

def write_phenotype_file_MIC(gemma_input_phenotype_file, phenotypes_found, isolates_with_phenotype, drug):
    
    """
    Convert input phenotype file into correct format for GEMMA and saves file.
    Adds an ancestral sequence and assumes it is sensitive
    
    Parameters
    ---------
    gemma_input_phenotype_file: str
        name of file to save
    
    phenotypes_found: pd.DataFrame
    
    isolates: pd.DataFrame
    
    """
    
    phenotypes = isolates_with_phenotype[["isolate_ID"]].merge(
        phenotypes_found[["ROLLINGDB_ID", drug]], right_on="ROLLINGDB_ID", left_on="isolate_ID", how="left"
    )
    print(len(phenotypes))
    phenotypes = phenotypes.drop_duplicates(subset=["isolate_ID"])
    phenotypes = phenotypes.drop_duplicates(subset=["ROLLINGDB_ID"])
    # Add the ancestor sequence, assuming ancestor is sensitive
    ancestor = pd.DataFrame({
        "isolate_ID": ["MT_H37Rv_anc"], 
        "ROLLINGDB_ID": [None], 
        drug: phenotypes[drug].min()
    })
    phenotypes = pd.concat([ancestor, phenotypes])
    print(len(phenotypes))
    phenotypes.iloc[:,-1].to_csv(gemma_input_phenotype_file, header=False, index=None, na_rep="NA")
    
    
def translate_genotypes(matrix, translator):
    """
    Translate matrix of genotypes into string characters. Used for converting Roger's numeric genotype 
    matrix back into strings of ATCG-
    
    Parameters
    ----------
    
    matrix: np.array 
        input genotype matrix
    translator: dict
        translator for converting entires of matrix to strings
    Returns
    -------
    np.array of str, translated matrix
    
    """
    
    translated_matrix = np.zeros_like(matrix, dtype='str')
    for i in range(matrix.shape[0]):
        translated_matrix[i, :] = [translator[x] for x in matrix[i,:]]
    return translated_matrix


# Add the ancestral sequence to the sequence matrix
def add_ancestor(mtb_ancestor_fasta, snps_to_keep, matrix, isolates_with_phenotype):
    mtb_ancestor = Alignment.from_file(open(mtb_ancestor_fasta, "r"))
    indices_to_slice = [x-1 for x in snps_to_keep.pos]
    ancestor_sequence = mtb_ancestor.matrix[:, indices_to_slice]

    matrix  = np.concatenate([ancestor_sequence, matrix])
    isolate_labels = ["MTB_ancestor"] + list(isolates_with_phenotype.isolate_ID)
    
    return matrix, isolate_labels
    
def write_phenotype_file(gemma_input_phenotype_file, phenotypes_found, isolates_with_phenotype):
    
    """
    Convert input phenotype file into correct format for GEMMA and saves file.
    Adds an ancestral sequence and assumes it is sensitive
    
    Parameters
    ---------
    gemma_input_phenotype_file: str
        name of file to save
    
    phenotypes_found: pd.DataFrame
    
    isolates: pd.DataFrame
    
    """
    
    phenotypes = isolates_with_phenotype[["isolate_ID"]].merge(
        phenotypes_found[["Isolate", kwargs['drug']]], left_on="isolate_ID", right_on="Isolate", how="left"
    )
    phenotypes = phenotypes.drop_duplicates(subset=["isolate_ID"])
    # Add the ancestor sequence, assuming ancestor is sensitive
    ancestor = pd.DataFrame({
        "Isolate": ["MT_H37Rv_anc"], 
        "isolate_ID": [None], 
        kwargs['drug']: "S"
    })
    phenotypes = pd.concat([ancestor, phenotypes])
    
    encoding = {"R": 1, "S": 0}
    phenotypes[kwargs['drug']] = [encoding[x] for x in phenotypes[kwargs['drug']]]

    phenotypes.iloc[:,-1].to_csv(gemma_input_phenotype_file, header=False, index=None, na_rep="NA")
    
def calculate_reference_allele_matrix_for_pair_regression(matrix, reference_alleles, encode_gaps=True, gap_code=np.nan,
        ref_allele_code=0, non_ref_allele_code=1):
    """
    Encodes a mapped genotype matrix with reference and non-reference allele codes
    NOTE: This function currently only works for 5-character (with gaps) alphabet

    ----------
    matrix : np.array
        N x L matrix containing N sequences of length L.
        Matrix must be mapped to range(0, num_symbols) using
        map_matrix function
    reference_alleles : np.array
        1 x L matrix with coding of reference allele for each position in L
    encode_gaps: boolean, optional (default=True)
        If True, gaps will be fille with gap_code. If False, gaps will be
        eliminated (ie, replaced with the major allele code)
    gap_code: str or numeric, optional (default=np.nan)
        encoding to use for gap characters if nogap=False
    ref_allele_code: str or numeric, optional (default=0)
        encoding to use for major alleles
    non_ref_allele_code: str or numeric, optional (default=1)
        encoding to use for minor alleles

    Returns
    -------
    np.array
        Matrix of size N x L with reference/non-reference allele encoding
        for all positions in L
    """

    if encode_gaps:
        gap_code=gap_code
        print("encoding gaps with gap code", gap_code)
    else:
        gap_code=ref_allele_code
        print("encoding gaps with reference allele", ref_allele_code)

    encoded_matrix = np.zeros(shape=matrix.shape, dtype=object)
    
    print(type(matrix))

    # For each column
    for i in np.arange(0, encoded_matrix.shape[1]):

        #grab that column
        vec = matrix[:, i]

        # create an encoded version of the column, with minor allele code as default
        encoded_vec = np.array([non_ref_allele_code] * len(vec), dtype=object)

        # If the row has a major allele, replace in encoded vec
        is_ref_allele = vec == reference_alleles[i]
        encoded_vec[is_ref_allele] = ref_allele_code

        # If the row has a gap character, replace in encoded vec
        is_gap = vec ==9
        encoded_vec[is_gap] = gap_code

        # save in the encoded matrix
        encoded_matrix[:, i] = encoded_vec

    return encoded_matrix

def calculate_reference_allele_matrix(matrix, reference_alleles, encode_gaps=True, gap_code=np.nan,
        ref_allele_code=0, non_ref_allele_code=1):
    """
    Encodes a mapped genotype matrix with reference and non-reference allele codes
    NOTE: This function currently only works for 5-character (with gaps) alphabet

    ----------
    matrix : np.array
        N x L matrix containing N sequences of length L.
        Matrix must be mapped to range(0, num_symbols) using
        map_matrix function
    reference_alleles : np.array
        1 x L matrix with coding of reference allele for each position in L
    encode_gaps: boolean, optional (default=True)
        If True, gaps will be fille with gap_code. If False, gaps will be
        eliminated (ie, replaced with the major allele code)
    gap_code: str or numeric, optional (default=np.nan)
        encoding to use for gap characters if nogap=False
    ref_allele_code: str or numeric, optional (default=0)
        encoding to use for major alleles
    non_ref_allele_code: str or numeric, optional (default=1)
        encoding to use for minor alleles

    Returns
    -------
    np.array
        Matrix of size N x L with reference/non-reference allele encoding
        for all positions in L
    """

    if encode_gaps:
        gap_code=gap_code
        print("encoding gaps with gap code", gap_code)
    else:
        gap_code=ref_allele_code
        print("encoding gaps with reference allele", ref_allele_code)


    encoded_matrix = np.zeros(shape=matrix.shape, dtype=object)

    # For each column
    for i in np.arange(0, encoded_matrix.shape[1]):

        #grab that column
        vec = matrix[:, i]

        # create an encoded version of the column, with minor allele code as default
        encoded_vec = np.array([non_ref_allele_code] * len(vec), dtype=object)

        # If the row has a major allele, replace in encoded vec
        is_ref_allele = np.equal(vec, reference_alleles[i])
        encoded_vec[is_ref_allele] = ref_allele_code

        # If the row has a gap character, replace in encoded vec
        is_gap = np.isnan(vec.astype(float))
        encoded_vec[is_gap] = gap_code

        # save in the encoded matrix
        encoded_matrix[:, i] = encoded_vec

    return encoded_matrix

def prepare_loci_file(matrix,
    isolates, snps,  output_loci_file, gap_code="NA", prefix=None):
    """
    Perpares input file for GEMMA
    Saves major and minor allele files if prefix is provided
    
    Parameters
    ----------
    matrix: np.array
        array of genotypes, N isolates x N positions
    isolates: iterable
        list of isolate names
    snps: iterable
        list of position names
    output_loci_file: str, optional (default=None)
        path to save the output file to
    gap_code: str, optional (default=NA)
        code with which to encode gaps in the GEMMA input loci file
    prefix: str, optional (default=None)
        output prefix for major allele and strains file to save
    """

    gm = GenotypeMatrix(
        isolates, 
        snps,         
        matrix_data = matrix
    )
    print("Made genotype matrix with shape", gm.matrix.shape)
    # write the input loci file
    of = open(output_loci_file, "w")
    print("writing Gemma with gap code", gap_code)
    gm.write_GEMMA(of, gap_code=gap_code)
    
    
    if prefix:
        # write the major allele file
        major_allele_file = f"{prefix}_major_alleles.csv"
        maj_alleles = pd.DataFrame(gm.major_alleles, index=gm.positions, columns=["major_alleles"])
        maj_alleles.loc[:,"major_alleles"] = [ALPHABET[x] for x in maj_alleles.major_alleles]
        maj_alleles.to_csv(major_allele_file)

        # write the strain file
        strain_file= f"{prefix}_strains.csv"
        strains = pd.DataFrame(gm.strains, columns=["strain"])
        strains.to_csv(strain_file, index=None)



In [2]:
drugs = ["AMIKACIN","CAPREOMYCIN","ETHAMBUTOL","ETHIONAMIDE","ISONIAZID","KANAMYCIN","MOXIFLOXACIN","PYRAZINAMIDE","RIFAMPICIN","STREPTOMYCIN"]

drug_to_abbrev = {    
    "AMIKACIN":"AMI",
     "CAPREOMYCIN":"CAP",
     "ETHAMBUTOL":"EMB",
     "ETHIONAMIDE":"ETA",
     "ISONIAZID":"INH",
     "KANAMYCIN":"KAN",
     "MOXIFLOXACIN":"MOXI",
     "PYRAZINAMIDE":"PZA",
     "RIFAMPICIN":"RIF",
     "STREPTOMYCIN":"STR"
}
kwargs={
    # "isolate_annotation_file":"/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/genotypes_isolate_annotation.pkl",
    # "genotype_array_file":"/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/genotypes_matrix.npy",
    # "snp_annotation_file":"/n/data1/hms/dbmi/farhat/anna/genome_logreg/variable_selection_20_09/snp_annotation_file_filtered.csv",
    # "phenotype_file":"/n/data1/hms/dbmi/farhat/anna/ABR_epistasis/prep_GEMMA_inputs/MIC_combined_data.csv",
    # "prefix":"gemma_test", 
    "gemma":"/n/data1/hms/dbmi/farhat/anna/software/gemma-0.98.1-linux-static",
#    "drug":"PYRAZINAMIDE",
#    "drug_abbrev": "PZA",
    "mtb_ancestor_fasta": "../input/MTB_ancestor_reference_modified.fasta",

}

## Part 2: Running for all MAF 0p001 loci


In [3]:
def compute_pve(drug, **kwargs):
    gemma_executable = kwargs["gemma"]
    pheno_file = f"MIC_runs/output/{drug}/{drug}.phenotypes"
    relatedness_loci_file = f"MIC_runs/output/{drug}/0.input_loci_files/MAF_0p001.loci"
    d_file = f"MIC_runs/output/{drug}/MAF_0p001.eigenD.txt"
    u_file = f"MIC_runs/output/{drug}/MAF_0p001.eigenU.txt"
    string = f"{gemma_executable} -g {relatedness_loci_file} -p {pheno_file} -d {d_file} -u {u_file} -maf 0 -miss 0.2 -lmm -notsnp -o {outfile}"
    process = string.split(" ")
    result = subprocess.run(process, stdout=subprocess.PIPE)

    stdout = result.stdout.decode('utf-8').split("\n")
    if len(stdout) > 8:
        out1 = stdout[8]
        out2 = stdout[9]
        assert 'pve estimate' in out1
        assert 'se(pve)' in out2
        pve = float(out1.split("=")[-1])
        se = float(out2.split("=")[-1])


In [4]:
table = []
#for drug in drugs:
for drug in ["AMIKACIN"]:
    print(drug)
    
    gemma_executable = kwargs["gemma"]
    pheno_file = f"output/{drug}/{drug}.phenotypes"
    relatedness_loci_file = f"output/{drug}/0.input_loci_files/MAF_0p001.loci"
    cxx_file = f"output/{drug}/MAF_0p001.cXX.txt"
    outfile = "heritability_temp.txt"
    string = f"{gemma_executable} -g {relatedness_loci_file} -p {pheno_file} -k {cxx_file} -maf 0 -miss 0.2 -lmm -notsnp -o {outfile}"
    print(string)
    process = string.split(" ")
    result = subprocess.run(process, stdout=subprocess.PIPE)

    stdout = result.stdout.decode('utf-8').split("\n")
    print(stdout)
    if len(stdout) > 8:
        out1 = stdout[9]
        out2 = stdout[10]
        assert 'pve estimate' in out1
        assert 'se(pve)' in out2
        pve = float(out1.split("=")[-1])
        se = float(out2.split("=")[-1])
        table.append([drug, pve, se])
    
    print(pve, se)
    
df = pd.DataFrame(table, columns=["drug", "pve", "se_pve"])
df.to_csv("PVE_all_MAF0p001.csv")   

AMIKACIN
/n/data1/hms/dbmi/farhat/anna/software/gemma-0.98.1-linux-static -g output/AMIKACIN/0.input_loci_files/MAF_0p001.loci -p output/AMIKACIN/AMIKACIN.phenotypes -k output/AMIKACIN/MAF_0p001.cXX.txt -maf 0 -miss 0.2 -lmm -notsnp -o heritability_temp.txt
['GEMMA 0.98.1 (2018-12-10) by Xiang Zhou and team (C) 2012-2018', 'error! fail to open phenotype file: output/AMIKACIN/AMIKACIN.phenotypes', 'error! fail to open mean genotype file: output/AMIKACIN/0.input_loci_files/MAF_0p001.loci', 'ERROR: Enforce failed for file_kin output/AMIKACIN/MAF_0p001.cXX.txt: open file in src/param.cpp at line 905 in CheckParam', '']


NameError: name 'pve' is not defined

## Part 2.5: Running for all homoplasic loci

In [7]:
# full_range = pd.read_csv("../output/05.single_mutations/snps_with_all_annotation.csv")
# full_range = list(full_range.pos)
# # reads in the SNP annotations
# snp_annotation = pd.read_pickle("/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/genotypes_SNP_annotation.pkl")
# snp_annotation['mat_index'] =snp_annotation.index
# print('annotation for N Snps:', len(snp_annotation))

# print("total number of SNPs", len(snp_annotation.query("pos in @full_range")))
# snps_subset = snp_annotation.query("pos in @full_range")   

# isolates = pd.read_pickle(kwargs['isolate_annotation_file'])

# for drug, drug_abbrev in drug_to_abbrev.items():
# #for drug, drug_abbrev in drug_to_a:
#     print(drug, drug_abbrev)
#     # Generate the cXX file
    
#     prefix = f"output/{drug}"
#     output_prefix = f"{drug}"
#     os.system(f"mkdir {prefix}")
#     os.system(f"mkdir {prefix}/0.input_loci_files")
#     os.system(f"mkdir {prefix}/1.job_submission_files")
#     os.system(f"mkdir {prefix}/2.run_output")

#     # get the isolates with phenotypes to keep
#     phenotypes = pd.read_csv(kwargs['phenotype_file'], index_col=0)
#     phenotypes_found_all = phenotypes.loc[~phenotypes[drug_abbrev].isnull(),:]
#     phenotypes_found = phenotypes_found_all[["ROLLINGDB_ID", f"{drug_abbrev}_midpoint"]]
#     phenotypes_found[drug] = np.log(phenotypes_found[f"{drug_abbrev}_midpoint"])
    
#     isolates = pd.read_pickle(kwargs['isolate_annotation_file'])
#     isolates_with_phenotype = isolates.query("isolate_ID in @phenotypes_found.ROLLINGDB_ID")
#     g = np.load("/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/genotypes_matrix.npy")
    
#     #### Prepare the phenotype input file ####
#     pheno_file = f"{prefix}/{drug}.phenotypes"
#     write_phenotype_file_MIC(pheno_file, phenotypes_found, isolates_with_phenotype, drug)

#     # slice the matrix for only isolates of interest
#     geno_mat = g
#     matrix = geno_mat[snps_subset.index, :][:, isolates_with_phenotype.index]
#     matrix = translate_genotypes(matrix, genotype_translator).T
#     print("Number of 9's in the matrix", np.sum(matrix=='-'))

    
#     # add the ancestral sequence
#     matrix, isolates = add_ancestor(kwargs["mtb_ancestor_fasta"], snps_subset, matrix, isolates_with_phenotype)
    
#     # Generate the locus file
#     relatedness_loci_file = f"{prefix}/0.input_loci_files/homoplasic_sites.loci"
#     print(f"Creating locus file for {drug} with N positions: {len(snps_subset.pos)}")
#     prepare_loci_file(matrix, isolates, list(snps_subset.pos), relatedness_loci_file, prefix=prefix)

#     cxx_file = f"{prefix}/homoplasic_sites.cXX.txt"

#     outfile = f"{output_prefix}/homoplasic_sites"
#     string1 = f"{kwargs['gemma']} -g {relatedness_loci_file} -p {pheno_file} -gk -maf 0 -o {outfile}"
#     print(string1)

#     # Run the eigen decomposition
#     outfile =f'{output_prefix}/homoplasic_sites'
#     string2 = f"{kwargs['gemma']} -g {relatedness_loci_file} -p {pheno_file} -k {cxx_file} -maf 0 -eigen -notsnp -o {outfile}"
#     print(string2)

#     print("creating GRM")
#     os.system(string1)

#     print("running eigendecomp")
#     os.system(string2)

In [8]:
# table = []
# for drug in drugs:
#     print(drug)
    
#     gemma_executable = kwargs["gemma"]
#     pheno_file = f"output/{drug}/{drug}.phenotypes"
#     relatedness_loci_file = f"output/{drug}/0.input_loci_files/homoplasic_sites.loci"
#     cxx_file = f"output/{drug}/homoplasic_sites.cXX.txt"
#     outfile = "heritability_temp.txt"
    
#     string = f"{gemma_executable} -g {relatedness_loci_file} -p {pheno_file} -k {cxx_file} -maf 0 -miss 0.2 -lmm -notsnp -o {outfile}"
#     print(string)
    
#     process = string.split(" ")
#     result = subprocess.run(process, stdout=subprocess.PIPE)

#     stdout = result.stdout.decode('utf-8').split("\n")
#     print(stdout)
#     if len(stdout) > 8:
#         out3 = stdout[7]
#         out1 = stdout[9]
#         out2 = stdout[10]
#         assert 'pve estimate' in out1
#         assert 'se(pve)' in out2
#         assert 'number of analyzed SNPs' in out3
#         pve = float(out1.split("=")[-1])
#         se = float(out2.split("=")[-1])
#         N_snps = float(out3.split("=")[-1])
#         table.append([drug, pve, se, N_snps])
    
#     print(pve, se)
    
# df = pd.DataFrame(table, columns=["drug", "pve", "se_pve", "N_snps"])
# df.to_csv("PVE_homoplasic_sites..csv")   

## Part 3: Running for all loci in antibiotic-associated genes



In [9]:
# drug_to_abbrev.items()

In [10]:
# full_range = pd.read_csv("../output/05.single_mutations/snps_with_all_annotation.csv", index_col=0)
# full_range = full_range.query("in_known_ABR_position")
# full_range["known_drug"] = [x.split(",") for x in full_range.known_drug]

# # reads in the SNP annotations
# snp_annotation = pd.read_pickle("/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/genotypes_SNP_annotation.pkl")
# snp_annotation['mat_index'] =snp_annotation.index
# print('annotation for N Snps:', len(snp_annotation))

# print("total number of SNPs", len(snp_annotation.query("pos in @full_range.pos")))
 
# isolates = pd.read_pickle(kwargs['isolate_annotation_file'])

# for drug, drug_abbrev in drug_to_abbrev.items():
#     if drug == "LEVOFLOXACIN":
#         continue
#     print(drug, drug_abbrev)
#     relevant_snps = full_range.query("@drug_abbrev in known_drug")
    
#     _keep = []
#     for idx, row in full_range.iterrows():
#         if drug_abbrev in row.known_drug:
#             _keep.append(idx)
            
#         elif drug_abbrev=='ETA':
#             if "ETH" in row.known_drug:
#                 _keep.append(idx)
#         elif drug_abbrev=='MOXI':
#             if "MXF" in row.known_drug:
#                 _keep.append(idx)
#         elif drug_abbrev=='STR':
#             if "STM" in row.known_drug:
#                 _keep.append(idx)
                
#     positions_found = list(set(full_range.loc[_keep].pos))
#     snps_subset = snp_annotation.query("pos in @positions_found")
    
#     print(f"found {len(positions_found)} resistance snps")
#     #break
#     # Generate the cXX file

#     prefix = f"output/{drug}"
#     output_prefix = f"{drug}"
#     os.system(f"mkdir {prefix}")
#     os.system(f"mkdir {prefix}/0.input_loci_files")
#     os.system(f"mkdir {prefix}/1.job_submission_files")
#     os.system(f"mkdir {prefix}/2.run_output")

#     # get the isolates with phenotypes to keep
#     phenotypes = pd.read_csv(kwargs['phenotype_file'], index_col=0)
#     phenotypes_found_all = phenotypes.loc[~phenotypes[drug_abbrev].isnull(),:]
#     phenotypes_found = phenotypes_found_all[["ROLLINGDB_ID", f"{drug_abbrev}_midpoint"]]
#     phenotypes_found[drug] = np.log(phenotypes_found[f"{drug_abbrev}_midpoint"])
    
#     isolates = pd.read_pickle(kwargs['isolate_annotation_file'])
#     isolates_with_phenotype = isolates.query("isolate_ID in @phenotypes_found.ROLLINGDB_ID")
#     g = np.load("/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/genotypes_matrix.npy")
    
#     #### Prepare the phenotype input file ####
#     pheno_file = f"{prefix}/{drug}.phenotypes"
#     write_phenotype_file_MIC(pheno_file, phenotypes_found, isolates_with_phenotype, drug)

#     # slice the matrix for only isolates of interest
#     geno_mat = g
#     matrix = geno_mat[snps_subset.index, :][:, isolates_with_phenotype.index]
#     matrix = translate_genotypes(matrix, genotype_translator).T
#     print("Number of 9's in the matrix", np.sum(matrix=='-'))

    
#     # add the ancestral sequence
#     matrix, isolates = add_ancestor(kwargs["mtb_ancestor_fasta"], snps_subset, matrix, isolates_with_phenotype)
    
#     # Generate the locus file
#     relatedness_loci_file = f"{prefix}/0.input_loci_files/ABR_sites.loci"
#     print(f"Creating locus file for {drug} with N positions: {len(snps_subset.pos)}")
#     prepare_loci_file(matrix, isolates, list(snps_subset.pos), relatedness_loci_file, prefix=prefix)

#     cxx_file = f"{prefix}/ABR_known.cXX.txt"

#     outfile = f"{output_prefix}/ABR_known"
#     string1 = f"{kwargs['gemma']} -g {relatedness_loci_file} -p {pheno_file} -gk -maf 0 -o {outfile}"
#     print(string1)

#     # Run the eigen decomposition
#     outfile =f'{output_prefix}/ABR_known'
#     string2 = f"{kwargs['gemma']} -g {relatedness_loci_file} -p {pheno_file} -k {cxx_file} -maf 0 -eigen -notsnp -o {outfile}"
#     print(string2)

#     print("creating GRM")
#     os.system(string1)

#     print("running eigendecomp")
#     os.system(string2)

In [11]:
# table = []
# for drug in drugs:
# #for drug in ["RIFAMPICIN"]:
#     print(drug)
    
#     gemma_executable = kwargs["gemma"]
#     pheno_file = f"output/{drug}/{drug}.phenotypes"
#     relatedness_loci_file = f"output/{drug}/0.input_loci_files/ABR_sites.loci"
#     cxx_file = f"output/{drug}/ABR_known.cXX.txt"
#     outfile = "heritability_temp.txt"
    
#     string = f"{gemma_executable} -g {relatedness_loci_file} -p {pheno_file} -k {cxx_file} -maf 0 -miss 0.2 -lmm -notsnp -o {outfile}"
#     print(string)
    
#     process = string.split(" ")
#     result = subprocess.run(process, stdout=subprocess.PIPE)

#     stdout = result.stdout.decode('utf-8').split("\n")
#     print(stdout)
#     if len(stdout) > 8:
#         out3 = stdout[7]
#         out1 = stdout[9]
#         out2 = stdout[10]
#         assert 'pve estimate' in out1
#         assert 'se(pve)' in out2
#         assert 'number of analyzed SNPs' in out3
#         pve = float(out1.split("=")[-1])
#         se = float(out2.split("=")[-1])
#         N_snps = float(out3.split("=")[-1])
#         table.append([drug, pve, se, N_snps])
    
#     print(pve, se)
    
# df = pd.DataFrame(table, columns=["drug", "pve", "se_pve", "N_snps"])
# df.to_csv("PVE_ABR_known.csv")   

## 3.5: Possible ABR sites

In [12]:
# full_range = pd.read_csv("../output/05.single_mutations/snps_with_all_annotation.csv")
# full_range = full_range.query("in_possible_ABR_gene")
# full_range["possible_drug"] = [x.split(",") for x in full_range.possible_drug]

# # reads in the SNP annotations
# snp_annotation = pd.read_pickle("/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/genotypes_SNP_annotation.pkl")
# snp_annotation['mat_index'] =snp_annotation.index
# print('annotation for N Snps:', len(snp_annotation))

# print("total number of SNPs", len(snp_annotation.query("pos in @full_range")))
# snps_subset = snp_annotation.query("pos in @full_range")   

# isolates = pd.read_pickle(kwargs['isolate_annotation_file'])

# for drug, drug_abbrev in drug_to_abbrev.items():
# #for drug, drug_abbrev in [("RIFAMPICIN", "RIF")]:
#     print(drug, drug_abbrev)
#     # Generate the cXX file
    
#     if drug == "LEVOFLOXACIN":
#         continue
    
#     _keep = []
#     for idx, row in full_range.iterrows():
#         if drug_abbrev in row.possible_drug:
#             _keep.append(idx)
#         elif drug_abbrev=='ETA':
#             if "ETH" in row.possible_drug:
#                 _keep.append(idx)
#         elif drug_abbrev=='MOXI':
#             if "MXF" in row.possible_drug:
#                 _keep.append(idx)
#         elif drug_abbrev=='STR':
#             if "STM" in row.possible_drug:
#                 _keep.append(idx)
#     positions_found = list(set(full_range.loc[_keep].pos))
#     snps_subset = snp_annotation.query("pos in @positions_found")
    
#     print(f"found {len(positions_found)} resistance snps")
#     prefix = f"output/{drug}"
#     output_prefix = f"{drug}"
#     os.system(f"mkdir {prefix}")
#     os.system(f"mkdir {prefix}/0.input_loci_files")
#     os.system(f"mkdir {prefix}/1.job_submission_files")
#     os.system(f"mkdir {prefix}/2.run_output")

#     # get the isolates with phenotypes to keep
#     phenotypes = pd.read_csv(kwargs['phenotype_file'], index_col=0)
#     phenotypes_found_all = phenotypes.loc[~phenotypes[drug_abbrev].isnull(),:]
#     phenotypes_found = phenotypes_found_all[["ROLLINGDB_ID", f"{drug_abbrev}_midpoint"]]
#     phenotypes_found[drug] = np.log(phenotypes_found[f"{drug_abbrev}_midpoint"])
    
#     isolates = pd.read_pickle(kwargs['isolate_annotation_file'])
#     isolates_with_phenotype = isolates.query("isolate_ID in @phenotypes_found.ROLLINGDB_ID")
#     g = np.load("/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/genotypes_matrix.npy")
    
#     #### Prepare the phenotype input file ####
#     pheno_file = f"{prefix}/{drug}.phenotypes"
#     write_phenotype_file_MIC(pheno_file, phenotypes_found, isolates_with_phenotype, drug)

#     # slice the matrix for only isolates of interest
#     geno_mat = g
#     matrix = geno_mat[snps_subset.index, :][:, isolates_with_phenotype.index]
#     matrix = translate_genotypes(matrix, genotype_translator).T
#     print("Number of 9's in the matrix", np.sum(matrix=='-'))

    
#     # add the ancestral sequence
#     matrix, isolates = add_ancestor(kwargs["mtb_ancestor_fasta"], snps_subset, matrix, isolates_with_phenotype)
    
#     # Generate the locus file
#     relatedness_loci_file = f"{prefix}/0.input_loci_files/ABR_possible.loci"
#     print(f"Creating locus file for {drug} with N positions: {len(snps_subset.pos)}")
#     prepare_loci_file(matrix, isolates, list(snps_subset.pos), relatedness_loci_file, prefix=prefix)

#     cxx_file = f"{prefix}/ABR_possible.cXX.txt"

#     outfile = f"{output_prefix}/ABR_possible"
#     string1 = f"{kwargs['gemma']} -g {relatedness_loci_file} -p {pheno_file} -gk -maf 0 -o {outfile}"
#     print(string1)

#     # Run the eigen decomposition
#     outfile =f'{output_prefix}/ABR_possible'
#     string2 = f"{kwargs['gemma']} -g {relatedness_loci_file} -p {pheno_file} -k {cxx_file} -maf 0 -eigen -notsnp -o {outfile}"
#     print(string2)

#     print("creating GRM")
#     os.system(string1)


In [6]:
# table = []
# for drug in drugs:
#     print(drug)
    
#     gemma_executable = kwargs["gemma"]
#     pheno_file = f"output/{drug}/{drug}.phenotypes"
#     relatedness_loci_file = f"output/{drug}/0.input_loci_files/ABR_possible.loci"
#     cxx_file = f"output/{drug}/ABR_possible.cXX.txt"
#     outfile = "heritability_temp.txt"
    
#     string = f"{gemma_executable} -g {relatedness_loci_file} -p {pheno_file} -k {cxx_file} -maf 0 -miss 0.2 -lmm -notsnp -o {outfile}"
#     print(string)
    
#     process = string.split(" ")
#     result = subprocess.run(process, stdout=subprocess.PIPE)

#     stdout = result.stdout.decode('utf-8').split("\n")
#     print(stdout)
#     if len(stdout) > 8:
#         out3 = stdout[7]
#         out1 = stdout[9]
#         out2 = stdout[10]
#         assert 'pve estimate' in out1
#         assert 'se(pve)' in out2
#         assert 'number of analyzed SNPs' in out3
#         pve = float(out1.split("=")[-1])
#         se = float(out2.split("=")[-1])
#         N_snps = float(out3.split("=")[-1])
#         table.append([drug, pve, se, N_snps])
    
#     print(pve, se)
    
# df = pd.DataFrame(table, columns=["drug", "pve", "se_pve", "N_snps"])
# df.to_csv("PVE_ABR_possible.csv")   

### Run for SNPs plus other sites that are epistatic (single sites only, subsequent and simultaneous hits)

In [7]:
# full_range = pd.read_csv("../output/05.single_mutations/snps_with_all_annotation.csv", index_col=0)
# full_range = full_range.query("in_known_ABR_position")
# full_range["known_drug"] = [x.split(",") for x in full_range.known_drug]

# # reads in the SNP annotations
# snp_annotation = pd.read_pickle("/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/genotypes_SNP_annotation.pkl")
# snp_annotation['mat_index'] =snp_annotation.index
# print('annotation for N Snps:', len(snp_annotation))


# isolates = pd.read_pickle(kwargs['isolate_annotation_file'])

# for drug, drug_abbrev in drug_to_abbrev.items():
#     print(drug, drug_abbrev)
#     # Generate the cXX file
#     _keep = []
#     for idx, row in full_range.iterrows():
#         if drug_abbrev in row.known_drug:
#             _keep.append(idx)
#         elif drug_abbrev=='ETA':
#             if "ETH" in row.known_drug:
#                 _keep.append(idx)
#         elif drug_abbrev=='MOXI':
#             if "MXF" in row.known_drug:
#                 _keep.append(idx)
#         elif drug_abbrev=='STR':
#             if "STM" in row.known_drug:
#                 _keep.append(idx)
                
#     positions_found = list(set(full_range.loc[_keep].pos))
    
#     antibiotic_hits = pd.read_csv(
#         f"/n/data1/hms/dbmi/farhat/anna/ABR_epistasis/DependentMutations2/output/04.antibiotic/{drug_abbrev}_antibiotic_hits.csv"
#     )
#     positions_found = positions_found + list(antibiotic_hits.position_j) + list(antibiotic_hits.position_i)
    
#     snps_subset = snp_annotation.query("pos in @positions_found")
    
#     prefix = f"output/{drug}"
#     output_prefix = f"{drug}"
#     os.system(f"mkdir {prefix}")
#     os.system(f"mkdir {prefix}/0.input_loci_files")
#     os.system(f"mkdir {prefix}/1.job_submission_files")
#     os.system(f"mkdir {prefix}/2.run_output")

#     # get the isolates with phenotypes to keep
#     phenotypes = pd.read_csv(kwargs['phenotype_file'], index_col=0)
#     phenotypes.replace("NA", np.nan, inplace=True)
    
#     phenotypes_found_all = phenotypes.loc[~phenotypes[drug_abbrev].isnull(),:]
#     phenotypes_found = phenotypes_found_all[["ROLLINGDB_ID", f"{drug_abbrev}_midpoint"]]
#     phenotypes_found[drug] = np.log(phenotypes_found[f"{drug_abbrev}_midpoint"])

#     isolates = pd.read_pickle(kwargs['isolate_annotation_file'])
#     isolates_with_phenotype = isolates.query("isolate_ID in @phenotypes_found.ROLLINGDB_ID")
#     g = np.load("/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/genotypes_matrix.npy")

# #     #### Prepare the phenotype input file ####
#     pheno_file = f"{prefix}/{drug}.phenotypes"
#     write_phenotype_file_MIC(pheno_file, phenotypes_found, isolates_with_phenotype, drug)

#     # slice the matrix for only isolates of interest
#     geno_mat = g
#     matrix = geno_mat[snps_subset.index, :][:, isolates_with_phenotype.index]
#     matrix = translate_genotypes(matrix, genotype_translator).T    

#     # add the ancestral sequence
#     matrix, isolates = add_ancestor(kwargs["mtb_ancestor_fasta"], snps_subset, matrix, isolates_with_phenotype)

# #     # Generate the locus file
    
#     relatedness_loci_file = f"{prefix}/0.input_loci_files/ABR_sites_plus_UK.loci"

#     print(f"Creating locus file for {drug} with N positions: {len(snps_subset.pos)}")

#     prepare_loci_file(matrix, isolates, list(snps_subset.pos), relatedness_loci_file, prefix=prefix)

#     cxx_file = f"{prefix}/heritability_test_ABR_UK.cXX.txt"

#     outfile = f"{output_prefix}/heritability_test_ABR_UK"
#     string1 = f"{kwargs['gemma']} -g {relatedness_loci_file} -p {pheno_file} -gk -maf 0 -o {outfile}"
#     print(string1)

#     # Run the eigen decomposition
#     outfile =f'{output_prefix}/heritability_test_ABR_UK'
#     string2 = f"{kwargs['gemma']} -g {relatedness_loci_file} -p {pheno_file} -k {cxx_file} -maf 0 -eigen -notsnp -o {outfile}"
#     print(string2)

#     print("creating GRM")
#     os.system(string1)

#     print("running eigendecomp")
#     os.system(string2)

In [8]:
# table = []
# for drug in drugs:
#     print(drug)
    
#     gemma_executable = kwargs["gemma"]
#     pheno_file = f"output/{drug}/{drug}.phenotypes"
#     relatedness_loci_file = f"output/{drug}/0.input_loci_files/ABR_sites_plus_UK.loci"
#     d_file = f"output/{drug}/heritability_test_ABR_UK.eigenD.txt"
#     u_file = f"output/{drug}/heritability_test_ABR_UK.eigenU.txt"
#     outfile = "heritability_temp.txt"
#     string = f"{gemma_executable} -g {relatedness_loci_file} -p {pheno_file} -d {d_file} -u {u_file} -maf 0 -miss 0.2 -lmm -notsnp -o {outfile}"
#     print(string)
#     process = string.split(" ")
#     result = subprocess.run(process, stdout=subprocess.PIPE)

#     stdout = result.stdout.decode('utf-8').split("\n")

#     print(stdout)
#     if len(stdout) > 8:
#         out3 = stdout[7]
#         out1 = stdout[8]
#         out2 = stdout[9]
#         assert 'pve estimate' in out1
#         assert 'se(pve)' in out2
#         assert 'number of analyzed SNPs' in out3
#         pve = float(out1.split("=")[-1])
#         se = float(out2.split("=")[-1])
#         N_snps = float(out3.split("=")[-1])
#         table.append([drug, pve, se, N_snps])
    
#     print(pve, se)
    
# df = pd.DataFrame(table, columns=["drug", "pve", "se_pve", "N_snps"])
# df.to_csv("PVE_ABR_UK_sites.csv")


### Run for POSSIBLE ANTIBIOTIC SNPs plus other sites that are epistatic (single sites only, subsequent and simultaneous hits)

In [9]:
# full_range = pd.read_csv("../output/05.single_mutations/snps_with_all_annotation.csv", index_col=0)
# full_range = full_range.query("in_possible_ABR_gene")
# full_range["possible_drug"] = [x.split(",") for x in full_range.possible_drug]

# # reads in the SNP annotations
# snp_annotation = pd.read_pickle("/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/genotypes_SNP_annotation.pkl")
# snp_annotation['mat_index'] =snp_annotation.index
# print('annotation for N Snps:', len(snp_annotation))


# isolates = pd.read_pickle(kwargs['isolate_annotation_file'])

# for drug, drug_abbrev in drug_to_abbrev.items():
#     print(drug, drug_abbrev)
#     # Generate the cXX file
#     _keep = []
#     for idx, row in full_range.iterrows():
#         if drug_abbrev in row.possible_drug:
#             _keep.append(idx)
#         elif drug_abbrev=='ETA':
#             if "ETH" in row.possible_drug:
#                 _keep.append(idx)
#         elif drug_abbrev=='MOXI':
#             if "MXF" in row.possible_drug:
#                 _keep.append(idx)
#         elif drug_abbrev=='STR':
#             if "STM" in row.possible_drug:
#                 _keep.append(idx)
                
#     positions_found = list(set(full_range.loc[_keep].pos))
    
#     antibiotic_hits = pd.read_csv(
#         f"/n/data1/hms/dbmi/farhat/anna/ABR_epistasis/DependentMutations2/output/04.antibiotic/{drug_abbrev}_antibiotic_hits.csv"
#     )
#     positions_found = positions_found + list(antibiotic_hits.position_j) + list(antibiotic_hits.position_i)
    
#     snps_subset = snp_annotation.query("pos in @positions_found")
    
#     prefix = f"output/{drug}"
#     output_prefix = f"{drug}"
#     os.system(f"mkdir {prefix}")
#     os.system(f"mkdir {prefix}/0.input_loci_files")
#     os.system(f"mkdir {prefix}/1.job_submission_files")
#     os.system(f"mkdir {prefix}/2.run_output")

#     # get the isolates with phenotypes to keep
#     phenotypes = pd.read_csv(kwargs['phenotype_file'], index_col=0)
#     phenotypes.replace("NA", np.nan, inplace=True)
    
#     phenotypes_found_all = phenotypes.loc[~phenotypes[drug_abbrev].isnull(),:]
#     phenotypes_found = phenotypes_found_all[["ROLLINGDB_ID", f"{drug_abbrev}_midpoint"]]
#     phenotypes_found[drug] = np.log(phenotypes_found[f"{drug_abbrev}_midpoint"])

#     isolates = pd.read_pickle(kwargs['isolate_annotation_file'])
#     isolates_with_phenotype = isolates.query("isolate_ID in @phenotypes_found.ROLLINGDB_ID")
#     g = np.load("/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/genotypes_matrix.npy")

# #     #### Prepare the phenotype input file ####
#     pheno_file = f"{prefix}/{drug}.phenotypes"
#     write_phenotype_file_MIC(pheno_file, phenotypes_found, isolates_with_phenotype, drug)

#     # slice the matrix for only isolates of interest
#     geno_mat = g
#     matrix = geno_mat[snps_subset.index, :][:, isolates_with_phenotype.index]
#     matrix = translate_genotypes(matrix, genotype_translator).T    

#     # add the ancestral sequence
#     matrix, isolates = add_ancestor(kwargs["mtb_ancestor_fasta"], snps_subset, matrix, isolates_with_phenotype)

# #     # Generate the locus file
    
#     relatedness_loci_file = f"{prefix}/0.input_loci_files/possible_ABR_sites_plus_UK.loci"

#     print(f"Creating locus file for {drug} with N positions: {len(snps_subset.pos)}")

#     prepare_loci_file(matrix, isolates, list(snps_subset.pos), relatedness_loci_file, prefix=prefix)

#     cxx_file = f"{prefix}/heritability_test_possible_ABR_UK.cXX.txt"

#     outfile = f"{output_prefix}/heritability_test_possible_ABR_UK"
#     string1 = f"{kwargs['gemma']} -g {relatedness_loci_file} -p {pheno_file} -gk -maf 0 -o {outfile}"
#     print(string1)

#     # Run the eigen decomposition
#     outfile =f'{output_prefix}/heritability_test_possible_ABR_UK'
#     string2 = f"{kwargs['gemma']} -g {relatedness_loci_file} -p {pheno_file} -k {cxx_file} -maf 0 -eigen -notsnp -o {outfile}"
#     print(string2)

#     print("creating GRM")
#     os.system(string1)

#     print("running eigendecomp")
#     os.system(string2)

In [10]:
# table = []
# for drug in drugs:
#     print(drug)
    
#     gemma_executable = kwargs["gemma"]
#     pheno_file = f"output/{drug}/{drug}.phenotypes"
#     relatedness_loci_file = f"output/{drug}/0.input_loci_files/possible_ABR_sites_plus_UK.loci"
#     d_file = f"output/{drug}/heritability_test_possible_ABR_UK.eigenD.txt"
#     u_file = f"output/{drug}/heritability_test_possible_ABR_UK.eigenU.txt"
#     outfile = "heritability_temp.txt"
#     string = f"{gemma_executable} -g {relatedness_loci_file} -p {pheno_file} -d {d_file} -u {u_file} -maf 0 -miss 0.2 -lmm -notsnp -o {outfile}"
#     print(string)
#     process = string.split(" ")
#     result = subprocess.run(process, stdout=subprocess.PIPE)

#     stdout = result.stdout.decode('utf-8').split("\n")

#     print(stdout)
#     if len(stdout) > 8:
#         out3 = stdout[7]
#         out1 = stdout[8]
#         out2 = stdout[9]
#         assert 'pve estimate' in out1
#         assert 'se(pve)' in out2
#         assert 'number of analyzed SNPs' in out3
#         pve = float(out1.split("=")[-1])
#         se = float(out2.split("=")[-1])
#         N_snps = float(out3.split("=")[-1])
#         table.append([drug, pve, se, N_snps])
    
#     print(pve, se)
    
# df = pd.DataFrame(table, columns=["drug", "pve", "se_pve", "N_snps"])
# df.to_csv("possible_PVE_ABR_UK_sites.csv")


### Run for SNPs + pairs that are in ABR genes or epistatic

In [6]:
full_range = pd.read_csv("../output/05.single_mutations/snps_with_all_annotation.csv", index_col=0)
full_range = full_range.query("in_possible_ABR_gene")
full_range["possible_drug"] = [x.split(",") for x in full_range.possible_drug]

# reads in the SNP annotations
snp_annotation = pd.read_pickle("/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/genotypes_SNP_annotation.pkl")
snp_annotation['mat_index'] =snp_annotation.index
print('annotation for N Snps:', len(snp_annotation))

isolates = pd.read_pickle(kwargs['isolate_annotation_file'])
#g = np.load("/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/genotypes_matrix.npy")

annotation for N Snps: 782565


In [15]:
for drug, drug_abbrev in drug_to_abbrev.items():
    
    ## 
    _keep = []
    for idx, row in full_range.iterrows():
        if drug_abbrev in row.possible_drug:
            _keep.append(idx)
        elif drug_abbrev=='ETA':
            if "ETH" in row.possible_drug:
                _keep.append(idx)
        elif drug_abbrev=='MOXI':
            if "MXF" in row.possible_drug:
                _keep.append(idx)
        elif drug_abbrev=='STR':
            if "STM" in row.possible_drug:
                _keep.append(idx)
                
    positions_found = list(set(full_range.loc[_keep].pos))
    
    antibiotic_hits = pd.read_csv(
        f"/n/data1/hms/dbmi/farhat/anna/ABR_epistasis/DependentMutations2/output/04.antibiotic/{drug_abbrev}_antibiotic_hits.csv"
    )
    positions_found = positions_found + list(antibiotic_hits.position_j) + list(antibiotic_hits.position_i)
    snps_subset = snp_annotation.query("pos in @positions_found")
    
    additional_sites = antibiotic_hits
    
    print(drug, drug_abbrev)
    # Generate the cXX file
    
    prefix = f"output/{drug}"
    output_prefix = f"{drug}"
    os.system(f"mkdir {prefix}")
    os.system(f"mkdir {prefix}/0.input_loci_files")
    os.system(f"mkdir {prefix}/1.job_submission_files")
    os.system(f"mkdir {prefix}/2.run_output")

    # get the isolates with phenotypes to keep
    phenotypes = pd.read_csv(kwargs['phenotype_file'], index_col=0)
    phenotypes.replace("NA", np.nan, inplace=True)
    
    phenotypes_found_all = phenotypes.loc[~phenotypes[drug_abbrev].isnull(),:]
    phenotypes_found = phenotypes_found_all[["ROLLINGDB_ID", f"{drug_abbrev}_midpoint"]]
    phenotypes_found[drug] = np.log(phenotypes_found[f"{drug_abbrev}_midpoint"])

    isolates = pd.read_pickle(kwargs['isolate_annotation_file'])
    isolates_with_phenotype = isolates.query("isolate_ID in @phenotypes_found.ROLLINGDB_ID")
    geno_mat = np.load("/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/genotypes_matrix.npy")

    #### Prepare the phenotype input file ####
    pheno_file = f"{prefix}/{drug}.phenotypes"
    write_phenotype_file_MIC(pheno_file, phenotypes_found, isolates_with_phenotype, drug)

    
    matrix = geno_mat[snps_subset.index, :][:, isolates_with_phenotype.index]
    
    #### Prepare the phenotype input file ####
    pheno_file = f"{prefix}/{drug}.phenotypes"
    write_phenotype_file_MIC(pheno_file, phenotypes_found, isolates_with_phenotype, drug)

    
    matrix = geno_mat[snps_subset.index, :][:, isolates_with_phenotype.index]
    
    ## Add the MTB ancestor sequence
    
    # read the ancestor sequence
    ancestral_seq_file="../input/MTB_ancestor_reference_modified.fasta"
    # subtract 1 or get wrecked. H37Rv positions are 1-indexed
    all_positions = [int(x) - 1 for x in snps_subset.pos]
    ancestral_seq = Alignment.from_file(open(ancestral_seq_file, "r"))
    ancestral_seq_subset = np.array([ancestral_seq.matrix[0,x] for x in all_positions])
    
    # convert to matrix encoding
    ancestral_seq_subset = np.array([genotype_translator_r[x] for x in list(ancestral_seq_subset)])
    
    # concatenate as FIRST sequence
    matrix = np.concatenate([ancestral_seq_subset.reshape(-1,1), matrix], axis=1)
    
    isolates = ["MTB_ANC"] + list(isolates_with_phenotype.isolate_ID)

    # add to strain list
    
    gm = GenotypeMatrix(
        position_list = snps_subset.pos,
        strain_list = isolates,
        matrix_data=matrix.T
    )
    
    print("Number of isolates", len(isolates))
    print("Number of SNPS", len(snps_subset))
    ## Convert to major allele matrix
    geno_mat = calculate_reference_allele_matrix_for_pair_regression(gm.matrix, ancestral_seq_subset) 
    print("shape of the genotype matrix", geno_mat.shape)
    snps_in_analysis = list(snps_subset.pos)
    ### To the matrix, add the pairs of SNPs
    
    matrix = deepcopy(geno_mat)
    for idx, row in additional_sites.iterrows():
        # lookup the index of the two positions
        pos_i = row.position_i
        pos_j = row.position_j
        
        idx_i = gm.position_to_index[pos_i]
        idx_j = gm.position_to_index[pos_j]
        
        new_row = np.multiply(geno_mat[:,idx_i], geno_mat[:, idx_j]).T
        matrix = np.concatenate([matrix, new_row.reshape(-1,1)], axis=1)
        
        snps_in_analysis.append(f"{pos_i}_{pos_j}")

# #     # Generate the locus file
    print("newly created cross matrix dimensions", matrix.shape)
    relatedness_loci_file = f"{prefix}/0.input_loci_files/ABR_sites_plus_all_dependent.loci"

    print(f"Creating locus file for {drug} with N positions: {len(snps_in_analysis)}")

    prepare_loci_file(matrix, isolates, snps_in_analysis, relatedness_loci_file)

    cxx_file = f"{prefix}/ABR_with_ABRpairs.cXX.txt"

    outfile = f"{output_prefix}/ABR_with_ABRpairs"
    string1 = f"{kwargs['gemma']} -g {relatedness_loci_file} -p {pheno_file} -gk -maf 0 -o {outfile}"
    print(string1)

    # Run the eigen decomposition
    outfile =f'{output_prefix}/ABR_with_ABRpairs'
    string2 = f"{kwargs['gemma']} -g {relatedness_loci_file} -p {pheno_file} -k {cxx_file} -maf 0 -eigen -notsnp -o {outfile}"
    print(string2)

    print("creating GRM")
    os.system(string1)

    print("running eigendecomp")
    os.system(string2)


AMIKACIN AMI


mkdir: cannot create directory ‘output/AMIKACIN’: File exists
mkdir: cannot create directory ‘output/AMIKACIN/0.input_loci_files’: File exists
mkdir: cannot create directory ‘output/AMIKACIN/1.job_submission_files’: File exists
mkdir: cannot create directory ‘output/AMIKACIN/2.run_output’: File exists
/home/ag300/anaconda3/envs/jupytervenv2/lib/python3.7/site-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1205
1206
1205
1206
Number of isolates 1206
Number of SNPS 179
encoding gaps with gap code nan
<class 'numpy.ndarray'>
shape of the genotype matrix (1206, 179)
newly created cross matrix dimensions (1206, 280)
Creating locus file for AMIKACIN with N positions: 280
Made genotype matrix with shape (1206, 280)
writing Gemma with gap code NA
calculating reference allele matrix with gap code NA
encoding gaps with gap code NA
<class 'numpy.ndarray'>
/n/data1/hms/dbmi/farhat/anna/software/gemma-0.98.1-linux-static -g output/AMIKACIN/0.input_loci_files/ABR_sites_plus_all_dependent.loci -p output/AMIKACIN/AMIKACIN.phenotypes -gk -maf 0 -o AMIKACIN/ABR_with_ABRpairs
/n/data1/hms/dbmi/farhat/anna/software/gemma-0.98.1-linux-static -g output/AMIKACIN/0.input_loci_files/ABR_sites_plus_all_dependent.loci -p output/AMIKACIN/AMIKACIN.phenotypes -k output/AMIKACIN/ABR_with_ABRpairs.cXX.txt -maf 0 -eigen -notsnp -o AMIKACIN/ABR_with_ABRpairs
creating GRM
GEMMA 0.98.1 (2018-12-10) by Xiang Zhou and team 

**** INFO: Done.


running eigendecomp
GEMMA 0.98.1 (2018-12-10) by Xiang Zhou and team (C) 2012-2018
Reading Files ... 
## number of total individuals = 1206
## number of analyzed individuals = 1206
## number of covariates = 1
## number of phenotypes = 1
## number of total SNPs/var        =      280
## number of analyzed SNPs         =      268
Start Eigen-Decomposition...


...

/home/ag300/anaconda3/envs/jupytervenv2/lib/python3.7/site-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1543
1489
1543
1489
Number of isolates 1489
Number of SNPS 391
encoding gaps with gap code nan
<class 'numpy.ndarray'>
shape of the genotype matrix (1489, 391)
newly created cross matrix dimensions (1489, 607)
Creating locus file for STREPTOMYCIN with N positions: 607
Made genotype matrix with shape (1489, 607)
writing Gemma with gap code NA
calculating reference allele matrix with gap code NA
encoding gaps with gap code NA
<class 'numpy.ndarray'>
/n/data1/hms/dbmi/farhat/anna/software/gemma-0.98.1-linux-static -g output/STREPTOMYCIN/0.input_loci_files/ABR_sites_plus_all_dependent.loci -p output/STREPTOMYCIN/STREPTOMYCIN.phenotypes -gk -maf 0 -o STREPTOMYCIN/ABR_with_ABRpairs
/n/data1/hms/dbmi/farhat/anna/software/gemma-0.98.1-linux-static -g output/STREPTOMYCIN/0.input_loci_files/ABR_sites_plus_all_dependent.loci -p output/STREPTOMYCIN/STREPTOMYCIN.phenotypes -k output/STREPTOMYCIN/ABR_with_ABRpairs.cXX.txt -maf 0 -eigen -notsnp -o STREPTOMYCIN/ABR_with_ABRpairs
creating GRM
GEMMA 0.9

**** INFO: Done.


running eigendecomp
GEMMA 0.98.1 (2018-12-10) by Xiang Zhou and team (C) 2012-2018
Reading Files ... 
## number of total individuals = 1489
## number of analyzed individuals = 1489
## number of covariates = 1
## number of phenotypes = 1
## number of total SNPs/var        =      607
## number of analyzed SNPs         =      599
Start Eigen-Decomposition...


**** WARNING: Matrix G has 1041 eigenvalues close to zero
**** INFO: Done.


In [16]:
table = []
for drug in drugs:
    print(drug)
    
    gemma_executable = kwargs["gemma"]
    pheno_file = f"output/{drug}/{drug}.phenotypes"
    relatedness_loci_file = f"output/{drug}/0.input_loci_files/ABR_sites_plus_all_dependent.loci"
    cxx_file = f"output/{drug}/ABR_with_ABRpairs.cXX.txt"
    outfile = "heritability_temp.txt"
    #string = f"{gemma_executable} -g {relatedness_loci_file}  -k {cxx_file} -p {pheno_file} -maf 0 -miss 0.2 -lmm -notsnp -o {outfile}"
    string = f"{gemma_executable} -g {relatedness_loci_file} -p {pheno_file} -k {cxx_file} -maf 0 -miss 0.2 -lmm -notsnp -o {outfile}"
    print(string)
    process = string.split(" ")
    result = subprocess.run(process, stdout=subprocess.PIPE)

    stdout = result.stdout.decode('utf-8').split("\n")
    print(stdout)
    
    print(stdout)
    if len(stdout) > 8:
        out3 = stdout[7]
        out1 = stdout[9]
        out2 = stdout[10]
        assert 'pve estimate' in out1
        assert 'se(pve)' in out2
        assert 'number of analyzed SNPs' in out3
        pve = float(out1.split("=")[-1])
        se = float(out2.split("=")[-1])
        N_snps = float(out3.split("=")[-1])
        table.append([drug, pve, se, N_snps])
    
    print(pve, se)
    
df = pd.DataFrame(table, columns=["drug", "pve", "se_pve", "N_snps"])
    
df.to_csv("PVE_ABR_dep_sites.csv")
df

AMIKACIN
/n/data1/hms/dbmi/farhat/anna/software/gemma-0.98.1-linux-static -g output/AMIKACIN/0.input_loci_files/ABR_sites_plus_all_dependent.loci -p output/AMIKACIN/AMIKACIN.phenotypes -k output/AMIKACIN/ABR_with_ABRpairs.cXX.txt -maf 0 -miss 0.2 -lmm -notsnp -o heritability_temp.txt


**** WARNING: Matrix G has 1009 eigenvalues close to zero
**** INFO: Done.


['GEMMA 0.98.1 (2018-12-10) by Xiang Zhou and team (C) 2012-2018', 'Reading Files ... ', '## number of total individuals = 1206', '## number of analyzed individuals = 1206', '## number of covariates = 1', '## number of phenotypes = 1', '## number of total SNPs/var        =      280', '## number of analyzed SNPs         =      268', 'Start Eigen-Decomposition...', 'pve estimate =0.565059', 'se(pve) =0.0408507', '                                                   0%\r================================================== 100%\r================================================== 100%\r', '']
['GEMMA 0.98.1 (2018-12-10) by Xiang Zhou and team (C) 2012-2018', 'Reading Files ... ', '## number of total individuals = 1206', '## number of analyzed individuals = 1206', '## number of covariates = 1', '## number of phenotypes = 1', '## number of total SNPs/var        =      280', '## number of analyzed SNPs         =      268', 'Start Eigen-Decomposition...', 'pve estimate =0.565059', 'se(pve) =0.04085

**** WARNING: Matrix G has 946 eigenvalues close to zero
**** INFO: Done.


['GEMMA 0.98.1 (2018-12-10) by Xiang Zhou and team (C) 2012-2018', 'Reading Files ... ', '## number of total individuals = 1102', '## number of analyzed individuals = 1102', '## number of covariates = 1', '## number of phenotypes = 1', '## number of total SNPs/var        =      215', '## number of analyzed SNPs         =      206', 'Start Eigen-Decomposition...', 'pve estimate =0.355128', 'se(pve) =0.0500529', '                                                   0%\r================================================== 100%\r================================================== 100%\r', '']
['GEMMA 0.98.1 (2018-12-10) by Xiang Zhou and team (C) 2012-2018', 'Reading Files ... ', '## number of total individuals = 1102', '## number of analyzed individuals = 1102', '## number of covariates = 1', '## number of phenotypes = 1', '## number of total SNPs/var        =      215', '## number of analyzed SNPs         =      206', 'Start Eigen-Decomposition...', 'pve estimate =0.355128', 'se(pve) =0.05005

...

['GEMMA 0.98.1 (2018-12-10) by Xiang Zhou and team (C) 2012-2018', 'Reading Files ... ', '## number of total individuals = 1522', '## number of analyzed individuals = 1522', '## number of covariates = 1', '## number of phenotypes = 1', '## number of total SNPs/var        =      405', '## number of analyzed SNPs         =      402', 'Start Eigen-Decomposition...', 'pve estimate =0.526072', 'se(pve) =0.0364663', '                                                   0%\r================================================== 100%\r================================================== 100%\r', '']
['GEMMA 0.98.1 (2018-12-10) by Xiang Zhou and team (C) 2012-2018', 'Reading Files ... ', '## number of total individuals = 1522', '## number of analyzed individuals = 1522', '## number of covariates = 1', '## number of phenotypes = 1', '## number of total SNPs/var        =      405', '## number of analyzed SNPs         =      402', 'Start Eigen-Decomposition...', 'pve estimate =0.526072', 'se(pve) =0.03646

**** WARNING: Matrix G has 1041 eigenvalues close to zero


['GEMMA 0.98.1 (2018-12-10) by Xiang Zhou and team (C) 2012-2018', 'Reading Files ... ', '## number of total individuals = 1489', '## number of analyzed individuals = 1489', '## number of covariates = 1', '## number of phenotypes = 1', '## number of total SNPs/var        =      607', '## number of analyzed SNPs         =      599', 'Start Eigen-Decomposition...', 'pve estimate =0.478958', 'se(pve) =0.0368205', '                                                   0%\r================================================== 100%\r================================================== 100%\r', '']
['GEMMA 0.98.1 (2018-12-10) by Xiang Zhou and team (C) 2012-2018', 'Reading Files ... ', '## number of total individuals = 1489', '## number of analyzed individuals = 1489', '## number of covariates = 1', '## number of phenotypes = 1', '## number of total SNPs/var        =      607', '## number of analyzed SNPs         =      599', 'Start Eigen-Decomposition...', 'pve estimate =0.478958', 'se(pve) =0.03682

**** INFO: Done.


,drug,pve,se_pve,N_snps
0,AMIKACIN,0.565059,0.040851,268.0
1,CAPREOMYCIN,0.355128,0.050053,206.0
2,ETHAMBUTOL,0.468603,0.038971,550.0
3,ETHIONAMIDE,0.230460,0.045134,145.0
4,ISONIAZID,0.568829,0.039509,403.0
5,KANAMYCIN,0.632181,0.040279,236.0
6,MOXIFLOXACIN,0.492332,0.064168,50.0
7,PYRAZINAMIDE,0.646346,0.052279,371.0
8,RIFAMPICIN,0.526072,0.036466,402.0
9,STREPTOMYCIN,0.478958,0.036820,599.0


# Use the antibiotic potentiating hits too

In [9]:
full_range = pd.read_csv("../output/05.single_mutations/snps_with_all_annotation.csv", index_col=0)
full_range = full_range.query("in_possible_ABR_gene")
full_range["possible_drug"] = [x.split(",") for x in full_range.possible_drug]

# reads in the SNP annotations
snp_annotation = pd.read_pickle("/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/genotypes_SNP_annotation.pkl")
snp_annotation['mat_index'] =snp_annotation.index
print('annotation for N Snps:', len(snp_annotation))

isolates = pd.read_pickle(kwargs['isolate_annotation_file'])
#g = np.load("/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/genotypes_matrix.npy")

for drug, drug_abbrev in drug_to_abbrev.items():
    
    ## 
    _keep = []
    for idx, row in full_range.iterrows():
        if drug_abbrev in row.possible_drug:
            _keep.append(idx)
        elif drug_abbrev=='ETA':
            if "ETH" in row.possible_drug:
                _keep.append(idx)
        elif drug_abbrev=='MOXI':
            if "MXF" in row.possible_drug:
                _keep.append(idx)
        elif drug_abbrev=='STR':
            if "STM" in row.possible_drug:
                _keep.append(idx)
                
    positions_found = list(set(full_range.loc[_keep].pos))
    
    antibiotic_hits = pd.read_csv(
        f"/n/data1/hms/dbmi/farhat/anna/ABR_epistasis/DependentMutations2/output/04.antibiotic/{drug_abbrev}_antibiotic_hits.csv"
    )
    antibiotic_hits1 = pd.read_csv(
        f"/n/data1/hms/dbmi/farhat/anna/ABR_epistasis/DependentMutations2/output/04.antibiotic/{drug_abbrev}_potentiator_antibiotic_hits.csv"
    )
    antibiotic_hits = pd.concat([antibiotic_hits, antibiotic_hits1])
    positions_found = positions_found + list(antibiotic_hits.position_j) + list(antibiotic_hits.position_i)
    snps_subset = snp_annotation.query("pos in @positions_found")
    
    additional_sites = antibiotic_hits
    
    print(drug, drug_abbrev)
    # Generate the cXX file
    
    prefix = f"output/{drug}"
    output_prefix = f"{drug}"
    os.system(f"mkdir {prefix}")
    os.system(f"mkdir {prefix}/0.input_loci_files")
    os.system(f"mkdir {prefix}/1.job_submission_files")
    os.system(f"mkdir {prefix}/2.run_output")

    # get the isolates with phenotypes to keep
    phenotypes = pd.read_csv(kwargs['phenotype_file'], index_col=0)
    phenotypes.replace("NA", np.nan, inplace=True)
    
    phenotypes_found_all = phenotypes.loc[~phenotypes[drug_abbrev].isnull(),:]
    phenotypes_found = phenotypes_found_all[["ROLLINGDB_ID", f"{drug_abbrev}_midpoint"]]
    phenotypes_found[drug] = np.log(phenotypes_found[f"{drug_abbrev}_midpoint"])

    isolates = pd.read_pickle(kwargs['isolate_annotation_file'])
    isolates_with_phenotype = isolates.query("isolate_ID in @phenotypes_found.ROLLINGDB_ID")
    geno_mat = np.load("/n/data1/hms/dbmi/farhat/Roger/homoplasy_project/rolling_DB_scrape/Genotypes_Filtered_2/genotypes_matrix.npy")

    #### Prepare the phenotype input file ####
    pheno_file = f"{prefix}/{drug}.phenotypes"
    write_phenotype_file_MIC(pheno_file, phenotypes_found, isolates_with_phenotype, drug)

    
    matrix = geno_mat[snps_subset.index, :][:, isolates_with_phenotype.index]
    
    #### Prepare the phenotype input file ####
    pheno_file = f"{prefix}/{drug}.phenotypes"
    write_phenotype_file_MIC(pheno_file, phenotypes_found, isolates_with_phenotype, drug)

    
    matrix = geno_mat[snps_subset.index, :][:, isolates_with_phenotype.index]
    
    ## Add the MTB ancestor sequence
    
    # read the ancestor sequence
    ancestral_seq_file="../input/MTB_ancestor_reference_modified.fasta"
    # subtract 1 or get wrecked. H37Rv positions are 1-indexed
    all_positions = [int(x) - 1 for x in snps_subset.pos]
    ancestral_seq = Alignment.from_file(open(ancestral_seq_file, "r"))
    ancestral_seq_subset = np.array([ancestral_seq.matrix[0,x] for x in all_positions])
    
    # convert to matrix encoding
    ancestral_seq_subset = np.array([genotype_translator_r[x] for x in list(ancestral_seq_subset)])
    
    # concatenate as FIRST sequence
    matrix = np.concatenate([ancestral_seq_subset.reshape(-1,1), matrix], axis=1)
    
    isolates = ["MTB_ANC"] + list(isolates_with_phenotype.isolate_ID)

    # add to strain list
    
    gm = GenotypeMatrix(
        position_list = snps_subset.pos,
        strain_list = isolates,
        matrix_data=matrix.T
    )
    
    print("Number of isolates", len(isolates))
    print("Number of SNPS", len(snps_subset))
    ## Convert to major allele matrix
    geno_mat = calculate_reference_allele_matrix_for_pair_regression(gm.matrix, ancestral_seq_subset) 
    print("shape of the genotype matrix", geno_mat.shape)
    snps_in_analysis = list(snps_subset.pos)
    ### To the matrix, add the pairs of SNPs
    
    matrix = deepcopy(geno_mat)
    for idx, row in additional_sites.iterrows():
        # lookup the index of the two positions
        pos_i = row.position_i
        pos_j = row.position_j
        
        idx_i = gm.position_to_index[pos_i]
        idx_j = gm.position_to_index[pos_j]
        
        new_row = np.multiply(geno_mat[:,idx_i], geno_mat[:, idx_j]).T
        matrix = np.concatenate([matrix, new_row.reshape(-1,1)], axis=1)
        
        snps_in_analysis.append(f"{pos_i}_{pos_j}")

# #     # Generate the locus file
    print("newly created cross matrix dimensions", matrix.shape)
    relatedness_loci_file = f"{prefix}/0.input_loci_files/ABR_potentiator_sites_plus_all_dependent.loci"

    print(f"Creating locus file for {drug} with N positions: {len(snps_in_analysis)}")

    prepare_loci_file(matrix, isolates, snps_in_analysis, relatedness_loci_file)

    cxx_file = f"{prefix}/ABR_with_ABRpairs_potentiator.cXX.txt"

    outfile = f"{output_prefix}/ABR_with_ABRpairs_potentiator"
    string1 = f"{kwargs['gemma']} -g {relatedness_loci_file} -p {pheno_file} -gk -maf 0 -o {outfile}"
    print(string1)

    # Run the eigen decomposition
    outfile =f'{output_prefix}/ABR_with_ABRpairs_potentiator'
    string2 = f"{kwargs['gemma']} -g {relatedness_loci_file} -p {pheno_file} -k {cxx_file} -maf 0 -eigen -notsnp -o {outfile}"
    print(string2)

    print("creating GRM")
    os.system(string1)

    print("running eigendecomp")
    os.system(string2)


annotation for N Snps: 782565
AMIKACIN AMI


mkdir: cannot create directory ‘output/AMIKACIN’: File exists
mkdir: cannot create directory ‘output/AMIKACIN/0.input_loci_files’: File exists
mkdir: cannot create directory ‘output/AMIKACIN/1.job_submission_files’: File exists
mkdir: cannot create directory ‘output/AMIKACIN/2.run_output’: File exists
/home/ag300/anaconda3/envs/jupytervenv2/lib/python3.7/site-packages/ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1205
1206
1205
1206
Number of isolates 1206
Number of SNPS 353
encoding gaps with gap code nan
<class 'numpy.ndarray'>
shape of the genotype matrix (1206, 353)
newly created cross matrix dimensions (1206, 793)
Creating locus file for AMIKACIN with N positions: 793
Made genotype matrix with shape (1206, 793)
writing Gemma with gap code NA
calculating reference allele matrix with gap code NA
encoding gaps with gap code NA
<class 'numpy.ndarray'>
/n/data1/hms/dbmi/farhat/anna/software/gemma-0.98.1-linux-static -g output/AMIKACIN/0.input_loci_files/ABR_potentiator_sites_plus_all_dependent.loci -p output/AMIKACIN/AMIKACIN.phenotypes -gk -maf 0 -o AMIKACIN/ABR_with_ABRpairs_potentiator
/n/data1/hms/dbmi/farhat/anna/software/gemma-0.98.1-linux-static -g output/AMIKACIN/0.input_loci_files/ABR_potentiator_sites_plus_all_dependent.loci -p output/AMIKACIN/AMIKACIN.phenotypes -k output/AMIKACIN/ABR_with_ABRpairs_potentiator.cXX.txt -maf 0 -eigen -notsnp -o AMIKACIN/ABR_with_ABRpairs_potentiator
cr

**** INFO: Done.


running eigendecomp
GEMMA 0.98.1 (2018-12-10) by Xiang Zhou and team (C) 2012-2018
Reading Files ... 
## number of total individuals = 1206
## number of analyzed individuals = 1206
## number of covariates = 1
## number of phenotypes = 1
## number of total SNPs/var        =      793
## number of analyzed SNPs         =      778
Start Eigen-Decomposition...


...

mkdir: cannot create directory ‘output/STREPTOMYCIN/0.input_loci_files’: File exists
mkdir: cannot create directory ‘output/STREPTOMYCIN/1.job_submission_files’: File exists
mkdir: cannot create directory ‘output/STREPTOMYCIN/2.run_output’: File exists
/home/ag300/anaconda3/envs/jupytervenv2/lib/python3.7/site-packages/ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1543
1489
1543
1489
Number of isolates 1489
Number of SNPS 595
encoding gaps with gap code nan
<class 'numpy.ndarray'>
shape of the genotype matrix (1489, 595)
newly created cross matrix dimensions (1489, 1200)
Creating locus file for STREPTOMYCIN with N positions: 1200
Made genotype matrix with shape (1489, 1200)
writing Gemma with gap code NA
calculating reference allele matrix with gap code NA
encoding gaps with gap code NA
<class 'numpy.ndarray'>
/n/data1/hms/dbmi/farhat/anna/software/gemma-0.98.1-linux-static -g output/STREPTOMYCIN/0.input_loci_files/ABR_potentiator_sites_plus_all_dependent.loci -p output/STREPTOMYCIN/STREPTOMYCIN.phenotypes -gk -maf 0 -o STREPTOMYCIN/ABR_with_ABRpairs_potentiator
/n/data1/hms/dbmi/farhat/anna/software/gemma-0.98.1-linux-static -g output/STREPTOMYCIN/0.input_loci_files/ABR_potentiator_sites_plus_all_dependent.loci -p output/STREPTOMYCIN/STREPTOMYCIN.phenotypes -k output/STREPTOMYCIN/ABR_with_ABRpairs_potentiator.cXX.txt -maf 0 -eigen -notsnp -o ST

**** INFO: Done.


running eigendecomp
GEMMA 0.98.1 (2018-12-10) by Xiang Zhou and team (C) 2012-2018
Reading Files ... 
## number of total individuals = 1489
## number of analyzed individuals = 1489
## number of covariates = 1
## number of phenotypes = 1
## number of total SNPs/var        =     1200
## number of analyzed SNPs         =     1192
Start Eigen-Decomposition...


**** WARNING: Matrix G has 625 eigenvalues close to zero
**** INFO: Done.


In [10]:
table = []
for drug in drugs:
    print(drug)
    
    gemma_executable = kwargs["gemma"]
    pheno_file = f"output/{drug}/{drug}.phenotypes"
    relatedness_loci_file = f"output/{drug}/0.input_loci_files/ABR_sites_plus_all_dependent.loci"
    cxx_file = f"output/{drug}/ABR_with_ABRpairs_potentiator.cXX.txt"
    outfile = "heritability_temp.txt"
    #string = f"{gemma_executable} -g {relatedness_loci_file}  -k {cxx_file} -p {pheno_file} -maf 0 -miss 0.2 -lmm -notsnp -o {outfile}"
    string = f"{gemma_executable} -g {relatedness_loci_file} -p {pheno_file} -k {cxx_file} -maf 0 -miss 0.2 -lmm -notsnp -o {outfile}"
    print(string)
    process = string.split(" ")
    result = subprocess.run(process, stdout=subprocess.PIPE)

    stdout = result.stdout.decode('utf-8').split("\n")
    print(stdout)
    
    print(stdout)
    if len(stdout) > 8:
        out3 = stdout[7]
        out1 = stdout[9]
        out2 = stdout[10]
        assert 'pve estimate' in out1
        assert 'se(pve)' in out2
        assert 'number of analyzed SNPs' in out3
        pve = float(out1.split("=")[-1])
        se = float(out2.split("=")[-1])
        N_snps = float(out3.split("=")[-1])
        table.append([drug, pve, se, N_snps])
    
    print(pve, se)
    
df = pd.DataFrame(table, columns=["drug", "pve", "se_pve", "N_snps"])
    
df.to_csv("PVE_ABR_dep_sites_with_potentiators.csv")
df

AMIKACIN
/n/data1/hms/dbmi/farhat/anna/software/gemma-0.98.1-linux-static -g output/AMIKACIN/0.input_loci_files/ABR_sites_plus_all_dependent.loci -p output/AMIKACIN/AMIKACIN.phenotypes -k output/AMIKACIN/ABR_with_ABRpairs_potentiator.cXX.txt -maf 0 -miss 0.2 -lmm -notsnp -o heritability_temp.txt


**** WARNING: Matrix G has 712 eigenvalues close to zero
**** INFO: Done.


['GEMMA 0.98.1 (2018-12-10) by Xiang Zhou and team (C) 2012-2018', 'Reading Files ... ', '## number of total individuals = 1206', '## number of analyzed individuals = 1206', '## number of covariates = 1', '## number of phenotypes = 1', '## number of total SNPs/var        =      793', '## number of analyzed SNPs         =      778', 'Start Eigen-Decomposition...', 'pve estimate =0.537309', 'se(pve) =0.0410739', '                                                   0%\r================================================== 100%\r================================================== 100%\r', '']
['GEMMA 0.98.1 (2018-12-10) by Xiang Zhou and team (C) 2012-2018', 'Reading Files ... ', '## number of total individuals = 1206', '## number of analyzed individuals = 1206', '## number of covariates = 1', '## number of phenotypes = 1', '## number of total SNPs/var        =      793', '## number of analyzed SNPs         =      778', 'Start Eigen-Decomposition...', 'pve estimate =0.537309', 'se(pve) =0.04107

**** WARNING: Matrix G has 688 eigenvalues close to zero
**** INFO: Done.


['GEMMA 0.98.1 (2018-12-10) by Xiang Zhou and team (C) 2012-2018', 'Reading Files ... ', '## number of total individuals = 1102', '## number of analyzed individuals = 1102', '## number of covariates = 1', '## number of phenotypes = 1', '## number of total SNPs/var        =      639', '## number of analyzed SNPs         =      620', 'Start Eigen-Decomposition...', 'pve estimate =0.407109', 'se(pve) =0.0529406', '                                                   0%\r================================================== 100%\r================================================== 100%\r', '']
['GEMMA 0.98.1 (2018-12-10) by Xiang Zhou and team (C) 2012-2018', 'Reading Files ... ', '## number of total individuals = 1102', '## number of analyzed individuals = 1102', '## number of covariates = 1', '## number of phenotypes = 1', '## number of total SNPs/var        =      639', '## number of analyzed SNPs         =      620', 'Start Eigen-Decomposition...', 'pve estimate =0.407109', 'se(pve) =0.05294

...

['GEMMA 0.98.1 (2018-12-10) by Xiang Zhou and team (C) 2012-2018', 'Reading Files ... ', '## number of total individuals = 1522', '## number of analyzed individuals = 1522', '## number of covariates = 1', '## number of phenotypes = 1', '## number of total SNPs/var        =      405', '## number of analyzed SNPs         =      402', 'Start Eigen-Decomposition...', 'pve estimate =0.588582', 'se(pve) =0.0380766', '                                                   0%\r================================================== 100%\r================================================== 100%\r', '']
['GEMMA 0.98.1 (2018-12-10) by Xiang Zhou and team (C) 2012-2018', 'Reading Files ... ', '## number of total individuals = 1522', '## number of analyzed individuals = 1522', '## number of covariates = 1', '## number of phenotypes = 1', '## number of total SNPs/var        =      405', '## number of analyzed SNPs         =      402', 'Start Eigen-Decomposition...', 'pve estimate =0.588582', 'se(pve) =0.03807

**** WARNING: Matrix G has 625 eigenvalues close to zero


['GEMMA 0.98.1 (2018-12-10) by Xiang Zhou and team (C) 2012-2018', 'Reading Files ... ', '## number of total individuals = 1489', '## number of analyzed individuals = 1489', '## number of covariates = 1', '## number of phenotypes = 1', '## number of total SNPs/var        =      607', '## number of analyzed SNPs         =      599', 'Start Eigen-Decomposition...', 'pve estimate =0.406087', 'se(pve) =0.041125', '                                                   0%\r================================================== 100%\r================================================== 100%\r', '']
['GEMMA 0.98.1 (2018-12-10) by Xiang Zhou and team (C) 2012-2018', 'Reading Files ... ', '## number of total individuals = 1489', '## number of analyzed individuals = 1489', '## number of covariates = 1', '## number of phenotypes = 1', '## number of total SNPs/var        =      607', '## number of analyzed SNPs         =      599', 'Start Eigen-Decomposition...', 'pve estimate =0.406087', 'se(pve) =0.041125

**** INFO: Done.


,drug,pve,se_pve,N_snps
0,AMIKACIN,0.537309,0.041074,778.0
1,CAPREOMYCIN,0.407109,0.052941,620.0
2,ETHAMBUTOL,0.453898,0.043955,885.0
3,ETHIONAMIDE,0.471663,0.055662,386.0
4,ISONIAZID,0.678160,0.033679,620.0
5,KANAMYCIN,0.621469,0.040641,744.0
6,MOXIFLOXACIN,0.598275,0.050873,50.0
7,PYRAZINAMIDE,0.622346,0.054800,371.0
8,RIFAMPICIN,0.588582,0.038077,402.0
9,STREPTOMYCIN,0.406087,0.041125,599.0
